# Web Scraping con Python
Vamos a usar las librerías [Requests](https://requests.kennethreitz.org/en/master/) y [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) para hacer web scraping. Instalación (en Anaconda):
```
conda install requests
conda install beautifulsoup4
```
o con pip:

```
pip install requests
pip install beautifulsoup4
```

## Librería Request

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
#r = requests.get("https://es.wikipedia.org/wiki/Web_scraping")
r = requests.get("https://idal.uv.es/ejemplo.html")

In [3]:
type(r)

requests.models.Response

In [4]:
r.url

'https://idal.uv.es/ejemplo.html'

Para obtener la respuesta completa (código HTML de la página) como una cadena de texto accedemos al atributo `text`:

In [5]:
print(r.text)

<!DOCTYPE html>
<html lang="es">
<head>
  <title>Página de ejemplo</title>
  <meta charset="UTF-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">
</head>
<body>

  <div class="jumbotron text-center">
    <h1>Página de ejemplo</h1>
    <p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p> 
  </div>
  
  <div class="container">
    <div class="row">
      <div class="col-sm-4">
        <h3>Columna 1</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
        <a class="btn" href="#" title="IDAL">Enlace 1</a>
      </div>
      <div class="col-sm-4">
        <h3>Columna 2</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>

El objeto devuelto es de un tipo específico con unos atributos determinados:

In [6]:
type(r)

requests.models.Response

In [7]:
[s for s in dir(r) if not s.startswith('_')]

['apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Podemos ver el código de respuesta del servidor (útil para detectar errores 4XX o 5XX):

In [8]:
r.status_code

200

Para comprobar el tipo de respuesta (Content Type) y ver si es HTML, JSON, XML, etc.:

In [9]:
r.headers["content-type"]

'text/html; charset=UTF-8'

Si el servidor diera la información de la codificación incorrecta (p. ej. ISO-8859-1 para una página UTF-8) podemos forzar una nueva codificación

In [10]:
r.encoding

'UTF-8'

In [11]:
#r.encoding='UTF-8' #cambiar a ISO-8859-1 para comprobar que muestra mal los acentos
r.encoding='UTF-8'
print(r.text)

<!DOCTYPE html>
<html lang="es">
<head>
  <title>Página de ejemplo</title>
  <meta charset="UTF-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">
</head>
<body>

  <div class="jumbotron text-center">
    <h1>Página de ejemplo</h1>
    <p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p> 
  </div>
  
  <div class="container">
    <div class="row">
      <div class="col-sm-4">
        <h3>Columna 1</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
        <a class="btn" href="#" title="IDAL">Enlace 1</a>
      </div>
      <div class="col-sm-4">
        <h3>Columna 2</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>

Podemos ver todas las cabeceras devueltas:

In [12]:
r.headers

{'Date': 'Wed, 30 Mar 2022 11:24:45 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Last-Modified': 'Wed, 10 Feb 2021 10:11:14 GMT', 'ETag': '"73d-5baf89f8467b1-gzip"', 'Accept-Ranges': 'bytes', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '673', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html; charset=UTF-8'}

O una cabecera en concreto

In [13]:
r.headers['Content-Type']

'text/html; charset=UTF-8'

### Extraer contenido del HTML
Usar expresiones regulares para buscar patrones HTML no está recomendado. Es mejor usar una librería para "parsear" el HTML y extraer el contenido de las etiquetas específicas.  
Aún así, las RegEx se pueden usar para buscar patrones específicos como listas de precios, direcciones de correo, números de teléfono.  
Podemos ejecutar una RegEx en el texto de respuesta para buscar un patrón específico, como p. ej. una URL:

In [14]:
import re

re.findall(r'https?://[\w_./]+', r.text)

['https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css',
 'https://idal.uv.es',
 'http://etse.uv.es',
 'http://uv.es']

## Librería BeautifulSoup
La librería BeautifulSoup se utiliza para extraer contenidos desde una página HTML. Tiene una API muy simple a la vez que potente.  
Para empezar, hay que convertir el HTML del texto de respuesta en una estructura DOM jerárquica que se pueda recorrer y buscar.


In [15]:
soup = BeautifulSoup(r.text, "html.parser")
print(soup)
print(soup.prettify()) # permite tener una visión más clara de la jerarquía de los elementos de la Web.

<!DOCTYPE html>

<html lang="es">
<head>
<title>Página de ejemplo</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css" rel="stylesheet"/>
</head>
<body>
<div class="jumbotron text-center">
<h1>Página de ejemplo</h1>
<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>
</div>
<div class="container">
<div class="row">
<div class="col-sm-4">
<h3>Columna 1</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 1</a>
</div>
<div class="col-sm-4">
<h3>Columna 2</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 2</a>
</div>
<div class="col-sm-4" id="col4">
<h3>Columna

In [19]:
type(soup)

bs4.BeautifulSoup

In [16]:
list(soup.children) # lista con 3 elementos

['html',
 '\n',
 <html lang="es">
 <head>
 <title>Página de ejemplo</title>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1" name="viewport"/>
 <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css" rel="stylesheet"/>
 </head>
 <body>
 <div class="jumbotron text-center">
 <h1>Página de ejemplo</h1>
 <p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>
 </div>
 <div class="container">
 <div class="row">
 <div class="col-sm-4">
 <h3>Columna 1</h3>
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
 <a class="btn" href="#" title="IDAL">Enlace 1</a>
 </div>
 <div class="col-sm-4">
 <h3>Columna 2</h3>
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
 <a class="btn" href="#" title="IDAL">Enlace 2</a>
 </div>
 <div class="col-sm

Los 3 elementos de la lista son del tipo siguiente: 

-El primero es un objeto `Doctype`, que contiene información sobre el tipo de documento.

-El segundo es un `NavigableString`, que representa texto encontrado en el documento HTML.

-El último elemento es un objeto `Tag`, que contiene otros tags anidados.


In [17]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

Escogemos el tag `html`y sus hijos tomando el tercer valor de la lista. Cada elemento de la lista devuelta por la propiedad de los hijos también es un objeto BeautifulSoup:

In [18]:
html = list(soup.children)[2]
list(html.children) # tenemos 2 tags, head y body

['\n',
 <head>
 <title>Página de ejemplo</title>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1" name="viewport"/>
 <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css" rel="stylesheet"/>
 </head>,
 '\n',
 <body>
 <div class="jumbotron text-center">
 <h1>Página de ejemplo</h1>
 <p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>
 </div>
 <div class="container">
 <div class="row">
 <div class="col-sm-4">
 <h3>Columna 1</h3>
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
 <a class="btn" href="#" title="IDAL">Enlace 1</a>
 </div>
 <div class="col-sm-4">
 <h3>Columna 2</h3>
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
 <a class="btn" href="#" title="IDAL">Enlace 2</a>
 </div>
 <div class="col-sm-4" id="col4">
 <h3

De esta manera podemos ir bajando en la estructura del HTLM. Si queremos información del body ...

In [19]:
body = list(html.children)[3]
#print(list(body.children))
p = list(body.children)[3]
print(p)

<div class="container">
<div class="row">
<div class="col-sm-4">
<h3>Columna 1</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 1</a>
</div>
<div class="col-sm-4">
<h3>Columna 2</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 2</a>
</div>
<div class="col-sm-4" id="col4">
<h3>Columna 3</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 3</a>
</div>
</div>
<div class="row" id="enlaces">
<h2>Algunos enlaces de prueba</h2>
<ul>
<li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>
<li><a class="enlace" href="http:/

In [20]:
p.get_text()

"\n\n\nColumna 1\nLorem ipsum dolor sit amet, consectetur adipisicing elit...\nUt enim ad minim veniam, quis nostrud exercitation ullamco laboris...\nEnlace 1\n\n\nColumna 2\nLorem ipsum dolor sit amet, consectetur adipisicing elit...\nUt enim ad minim veniam, quis nostrud exercitation ullamco laboris...\nEnlace 2\n\n\nColumna 3\nLorem ipsum dolor sit amet, consectetur adipisicing elit...\nUt enim ad minim veniam, quis nostrud exercitation ullamco laboris...\nEnlace 3\n\n\n\nAlgunos enlaces de prueba\n\nIntelligent Data Analysis Laboratory\nEscola Tècnica Superior d'Enginyeria\nUniversitat de València\n\n\n"

Esta forma de acceder a los diferentes elementos necesita demasiados comandos. Vamos a ver cómo parsear el documento HTLM de manera más sencilla.

#### Elementos del DOM del objeto soup

Podemos referirnos a cada elemento del DOM por su nombre dentro del objeto `soup` (si hay varias apariciones sólo devuelve la primera, como por ejemplo los hipervínculos)

In [21]:
soup.title

<title>Página de ejemplo</title>

In [22]:
titulo = soup.title.text
print(titulo)

Página de ejemplo


In [22]:
soup.a

<a class="btn" href="#" title="IDAL">Enlace 1</a>

In [23]:
soup.a['class']

['btn']

El método `find_all` devuelve una lista con todas las etiquetas que encuentra. El método `find` sólo devuelve el primer resultado de la búsqueda.   
En ambos casos, cada etiqueta contiene todo su contenido en la estructura DOM.

In [24]:
link = soup.find("a")
link

<a class="btn" href="#" title="IDAL">Enlace 1</a>

De cada enlace podemos obtener sus atributos

In [22]:
link.get('class') #equivalente a link['class']

['btn']

In [23]:
link.attrs

{'class': ['btn'], 'href': '#', 'title': 'IDAL'}

Podemos encadenar varias etiquetas a buscar (p=párrafo, em=texto enfatizado).

In [32]:
soup.find('p').find('em')

<em>HTML</em>

Podemos acotar la búsqueda a una clase específica de la etiqueta (p.ej. `<a class="enlace">...</a>`)

EL objeto que devuelve `find()` es del tipo `Tag`


In [28]:
type(tag)

bs4.element.Tag

In [25]:
tag = soup.find("a", class_="enlace")
print(tag)

<a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>


In [26]:
#o simplemente
tag = soup.find("a", "enlace")
print(tag)

<a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>


Podemos buscar una etiqueta con un atributo ID específico (p.ej.: `<div id="col4">...</div>`)

In [27]:
tag = soup.find("div", id="col4")
print(tag)

<div class="col-sm-4" id="col4">
<h3>Columna 3</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 3</a>
</div>


In [29]:
#atributos y métodos de Tag
[s for s in dir(tag) if not s.startswith('_')]

['append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'decomposed',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings',
 'fetchParents',
 'fetchPrevious',
 'fetchPreviousSiblings',
 'find',
 'findAll',
 'findAllNext',
 'findAllPrevious',
 'findChild',
 'findChildren',
 'findNext',
 'findNextSibling',
 'findNextSiblings',
 'findParent',
 'findParents',
 'findPrevious',
 'findPreviousSibling',
 'findPreviousSiblings',
 'find_all',
 'find_all_next',
 'find_all_previous',
 'find_next',
 'find_next_sibling',
 'find_next_siblings',
 'find_parent',
 'find_parents',
 'find_previous',
 'find_previous_sibling',
 'find_previous_siblings',
 'format_string',
 'formatter_for_name',
 'get',
 'getText',
 'get_attribute_list',
 'get_text',
 'has_attr',
 'has_key',
 'hidden',
 'index',
 'insert',
 'insert_after',
 'insert_before',
 'isSelfClosing'

Para buscar todas las etiquetas de un tipo en el árbol DOM (por ejemplo los enlaces `<a>` de una página) 

In [30]:
soup.find_all("a")

[<a class="btn" href="#" title="IDAL">Enlace 1</a>,
 <a class="btn" href="#" title="IDAL">Enlace 2</a>,
 <a class="btn" href="#" title="IDAL">Enlace 3</a>,
 <a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>,
 <a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a>,
 <a class="enlace" href="http://uv.es" title="UV">Universitat de València</a>]

In [33]:
type(soup.find_all("a")[0])

bs4.element.Tag

In [34]:
#hay un shortcut para find_all
soup("a")

[<a class="btn" href="#" title="IDAL">Enlace 1</a>,
 <a class="btn" href="#" title="IDAL">Enlace 2</a>,
 <a class="btn" href="#" title="IDAL">Enlace 3</a>,
 <a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>,
 <a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a>,
 <a class="enlace" href="http://uv.es" title="UV">Universitat de València</a>]

In [35]:
soup.find_all(["a", "p"])

[<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>,
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>,
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>,
 <a class="btn" href="#" title="IDAL">Enlace 1</a>,
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>,
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>,
 <a class="btn" href="#" title="IDAL">Enlace 2</a>,
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>,
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>,
 <a class="btn" href="#" title="IDAL">Enlace 3</a>,
 <a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>,
 <a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a>,
 <a class="enlace" href="http://uv.es" title="UV">Universitat de València</a>]

In [36]:
soup.find_all(re.compile("h\d+"))

[<h1>Página de ejemplo</h1>,
 <h3>Columna 1</h3>,
 <h3>Columna 2</h3>,
 <h3>Columna 3</h3>,
 <h2>Algunos enlaces de prueba</h2>]

Podemos filtrar por un atributo del Tag (por defecto es la clase si no indicamos otra cosa)

In [37]:
for link in soup.find_all('a', class_="enlace"):
    print(link)

<a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>
<a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a>
<a class="enlace" href="http://uv.es" title="UV">Universitat de València</a>


También podemos obtener los atributos de cada uno de los elementos de una lista de etiquetas obtenida con `find_all` o `select`

In [43]:
for link in soup.find_all('a', class_="enlace"):
    print(link["href"])

https://idal.uv.es
http://etse.uv.es
http://uv.es


In [44]:
enlaces = soup.find_all("a", "btn")
for e in enlaces:
    print(e)

<a class="btn" href="#" title="IDAL">Enlace 1</a>
<a class="btn" href="#" title="IDAL">Enlace 2</a>
<a class="btn" href="#" title="IDAL">Enlace 3</a>


Podemos hacer una búsqueda de otra etiqueta anidada dentro de la primera búsqueda (útil por ejemplo para buscar elementos genéricos dentro de una sección específica de la página)

In [40]:
tags = soup.find("div", id="col4").find_all("a")
print(tags)

[<a class="btn" href="#" title="IDAL">Enlace 3</a>]


Repetimos esta búsqueda especificando un selector CSS (más simple que lo anterior si se conoce la sintaxis CSS)

In [41]:
tags = soup.select("#col4 a")
print(tags)

[<a class="btn" href="#" title="IDAL">Enlace 3</a>]


In [42]:
type(tags)

bs4.element.ResultSet

El atributo `contents` del resultado de una etiqueta individual (objeto `bs4.element.Tag`) contiene una lista de objetos con su contenido interno (que incluye tanto los nodos de texto como la representación y texto de todo el HTML anidado).  
Si el objeto sólo contiene texto es del tipo `NavigableString`, y si contiene etiqueta es del tipo `Tag`. Los saltos de línea (`\n`) entre las etiquetas se consideran elementos de texto y aparecen en la lista devuelta.

In [43]:
print(soup.find("title"))

<title>Página de ejemplo</title>


In [44]:
inner_contents = soup.find("title").contents
print(inner_contents)

['Página de ejemplo']


In [45]:
soup.find("div", id="col4")

<div class="col-sm-4" id="col4">
<h3>Columna 3</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 3</a>
</div>

In [46]:
inner_contents = soup.find("div", id="col4").contents
print(inner_contents)

['\n', <h3>Columna 3</h3>, '\n', <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>, '\n', <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>, '\n', <a class="btn" href="#" title="IDAL">Enlace 3</a>, '\n']


In [47]:
inner_contents[0]

'\n'

In [46]:
type(inner_contents[0])

bs4.element.NavigableString

In [47]:
inner_contents[1]

<h3>Columna 3</h3>

In [48]:
type(inner_contents[1])

bs4.element.Tag

El atributo `text` del objeto `bs4.element.Tag` contiene una lista de strings con los textos contenidos en la etiqueta, ignorando todas las etiquetas HTML (como por ejemplo `<span>`, `<strong>` o `<i>`):

In [49]:
inner_text = soup.find("div", id="col4").text.strip()
print(inner_text)

Columna 3
Lorem ipsum dolor sit amet, consectetur adipisicing elit...
Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...
Enlace 3


Aquí el método `strip()` simplemente elimina las líneas en blanco (`\n`) del resultado.  

### Búsqueda por función
Podemos definir una función booleana para hacer la búsqueda de elementos en el árbol DOM

In [50]:
def etiquetas(tag):
    return tag.has_attr('id')

soup.find_all(etiquetas)

[<div class="col-sm-4" id="col4">
 <h3>Columna 3</h3>
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
 <a class="btn" href="#" title="IDAL">Enlace 3</a>
 </div>,
 <div class="row" id="enlaces">
 <h2>Algunos enlaces de prueba</h2>
 <ul>
 <li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>
 <li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>
 <li><a class="enlace" href="http://uv.es" title="UV">Universitat de València</a></li>
 </ul>
 </div>]

In [49]:
[(t.name, t.attrs) for t in soup.find_all(etiquetas)]

[('div', {'class': ['col-sm-4'], 'id': 'col4'}),
 ('div', {'class': ['row'], 'id': 'enlaces'})]

## Navegación por la estructura DOM
Cada elemento `Tag` guarda información de su posición dentro del árbol de etiquetas HTML del documento (estructura DOM) de manera que se puede navegar desde cada etiqueta a sus etiquetas relacionadas:
### Navegar hacia abajo
Podemos usar los nombres de las etiquetas como atributos encadenados del objeto `soup` para recorrer el árbol hacia abajo:

In [51]:
soup.div.h1 #busca la primera etiqueta <h1> dentro de la primera etiqueta <div> 

<h1>Página de ejemplo</h1>

In [54]:
soup("div")[1].h3

<h3>Columna 1</h3>

Todos los hijos de una etiqueta están en su atributo `contents`:

In [48]:
soup.div.contents

['\n',
 <h1>Página de ejemplo</h1>,
 '\n',
 <p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>,
 '\n']

Las etiquetas que sólo contienen texto aparecen en la lista de `contents` como un objeto `NavigableString`

In [55]:
soup.div.h1.contents

['Página de ejemplo']

In [56]:
type(soup.div.h1.contents[0])

bs4.element.NavigableString

Podemos iterar sobre los descendientes de una etiqueta con el iterador `children`:

In [57]:
soup.div.children

In [58]:
[child for child in soup.div.children]

['\n',
 <h1>Página de ejemplo</h1>,
 '\n',
 <p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>,
 '\n']

In [59]:
#la lista que genera children es equivalente a contents
list(soup.div.children)==soup.div.contents

True

Con `contents` y `children` consideran sólo los descencientes directos de una etiqueta. Con `descendants` accedemos iterativamente a todos sus descendientes:

In [60]:
soup.div.p

<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>

In [61]:
soup.div.p

<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>

In [62]:
soup.div.p.descendants

<generator object Tag.descendants at 0x7f943b7655d0>

In [63]:
[child for child in soup.div.p.descendants]

['Página ',
 <em>HTML</em>,
 'HTML',
 ' con texto de ejemplo para hacer web scraping']

Para obtener sólo el texto de una etiqueta usamos `string` o para obtener iterativamente todos los textos contenidos usamos el iterador `strings`:

In [64]:
soup.div.h1.string

'Página de ejemplo'

In [65]:
#si una etiqueta contiene otras etiquetas el atributo string está vacío
soup.div.p

<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>

In [66]:
soup.div.p.string

In [67]:
#iteramos por todos los elementos contenidos para obtener su string
[s for s in soup.div.p.strings]

['Página ', 'HTML', ' con texto de ejemplo para hacer web scraping']

### Navegar hacia arriba
Con `parent` obtenemos la etiqueta superior a una dada, y con `parents` obtenemos iterativamente todos sus ascendientes:

In [68]:
soup.div.h1.parent

<div class="jumbotron text-center">
<h1>Página de ejemplo</h1>
<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>
</div>

In [69]:
soup.find("div", id="col4").parent

<div class="row">
<div class="col-sm-4">
<h3>Columna 1</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 1</a>
</div>
<div class="col-sm-4">
<h3>Columna 2</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 2</a>
</div>
<div class="col-sm-4" id="col4">
<h3>Columna 3</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 3</a>
</div>
</div>

In [70]:
for p in soup.find("div", id="col4").parents:
    print(p.name, p.attrs)

div {'class': ['row']}
div {'class': ['container']}
body {}
html {'lang': 'es'}
[document] {}


### Navegar hacia los lados
Los métodos `next_sibling` y `previous_sibling` acceden a la etiqueta posterior o anterior del mismo nivel. Los métodos `next_siblings` y `previous_siblings` lo hacen iterativamente

In [71]:
soup.find("div", id="enlaces")

<div class="row" id="enlaces">
<h2>Algunos enlaces de prueba</h2>
<ul>
<li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>
<li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>
<li><a class="enlace" href="http://uv.es" title="UV">Universitat de València</a></li>
</ul>
</div>

In [72]:
soup.find("div", id="enlaces").ul.contents

['\n',
 <li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>,
 '\n',
 <li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>,
 '\n',
 <li><a class="enlace" href="http://uv.es" title="UV">Universitat de València</a></li>,
 '\n']

In [73]:
soup.find("div", id="enlaces").li.previous_sibling

'\n'

In [74]:
soup.find("div", id="enlaces").li.next_sibling.next_sibling

<li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>

In [75]:
soup.find("div", id="enlaces").find("a",{"title": "ETSE"}).parent

<li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>

In [76]:
#buscamos elementos hermanos a continuación del <li> intermedio
enlace_etse = soup.find("div", id="enlaces").find("a",{"title": "ETSE"})
[tag for tag in enlace_etse.parent.next_siblings]

['\n',
 <li><a class="enlace" href="http://uv.es" title="UV">Universitat de València</a></li>,
 '\n']

In [77]:
#buscamos elementos hermanos anteriores al <li> final

[tag for tag in soup.find("div", id="enlaces").find("a",{"title": "UV"}).parent.previous_siblings]

['\n',
 <li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>,
 '\n',
 <li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>,
 '\n']

### Ejemplo completo
Del siguiente código HTML vamos a extraer los ítems listados y su precio

In [23]:
texto = """
<body>
<div id="listings_prices">
 <div class="item">
  <li class="item_name">Watch</li>
  <div class="main_price">Price: $66.68</div>
  <div class="discounted_price">Discounted price: $46.68</div>
   </div>
   <div class="item">
  <li class="item_name">Watch2</li>
  <div class="main_price">Price: $56.68</div>
   </div>
</div>
</body>
"""

In [24]:
soup = BeautifulSoup(texto, "html.parser")
for product in soup.find_all("div", "item"):
    product_title = product.find("li").text 
    product_price = re.search(r'\$\d+\.\d+', product.find("div", "main_price").text)[0]
    print(f"{product_title} is selling for {product_price}")

Watch is selling for $66.68
Watch2 is selling for $56.68
